In [1]:
ls

data-sample.txt*  __init__.py   main.py      sample.pk  Untitled.ipynb
definitions/      __init__.pyc  main.pyc     tags
dev.pk            learning/     processing/  test.pk
evaluation/       LICENSE.txt   README.md    train.pk


python learning/OieInduction.py --pickled_dataset sample.pk --model_name discrete-autoencoder --model AC --optimization 1 --epochs 10 --batch_size 100 --relations_number 10 --negative_samples_number 5 --l2_regularization 0.1 --alpha 0.1 --seed 2 --embed_size 10 --learning_rate 0.1


help funcs 

In [22]:
def print_dict(tmp_dict):
    for item in tmp_dict:
        print "{:30s} : {:10s}".format(item, str(tmp_dict[item]))

In [2]:
import argparse
import os

import numpy as np

import sys
import time
import cPickle as pickle
import operator
from theano import sparse
import theano
import theano.tensor as T
from learning.OieModel import OieModelFunctions

from learning.OieData import DataSetManager
from learning.OieData import MatrixDataSet
from processing.OiePreprocessor import FeatureLexicon
from evaluation.OieEvaluation import singleLabelClusterEvaluation
import definitions.settings as settings
from learning.NegativeExampleGenerator import NegativeExampleGenerator
from collections import OrderedDict

['', '/home/yliangav/anaconda3/envs/py27/lib/python27.zip', '/home/yliangav/anaconda3/envs/py27/lib/python2.7', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/plat-linux2', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/lib-tk', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/lib-old', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/lib-dynload', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/site-packages', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/site-packages/Sphinx-1.5.6-py2.7.egg', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/yliangav/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/extensions', '/home/yliangav/.ipython']


In [3]:
from learning.OieInduction import loadData

# dataset

 dataset format

The file must be tab-separated an with the following fields:

 - lexicalized dependency path between arguments (entities) of the relation,  
 - first entity  
 - second entity  
 - entity types of the first and second entity  
 - trigger word  
 - id of the sentence  
 - raw sentence  
 - pos tags of the entire sentence  
 - relation between the two entities if any (used only for evaluation)

The dataset pickle file format:
```python
pickle.dump(featureExtrs, pklFile, protocol=pklProtocol)

pickle.dump(relationLexicon, pklFile, protocol=pklProtocol)

pickle.dump(dataset, pklFile, protocol=pklProtocol)

pickle.dump(goldstandard, pklFile, protocol=pklProtocol)
```

In [42]:
def getFeatures(lexicon, featureExs, info, arg1=None, arg2=None, expand=False):
    """
    getFeatures(relationLexicon, featureExtrs, [re[1], re[4], re[5], re[7], re[8], re[6]],
                                                             re[2], re[3], True)
    :param lexicon: FeatureLexicon instance, init empty(default constructor) 
    :param featureExs: feature exactors
    :param info: features except entity
    :param arg1: entity1
    :param arg2: entity2
    :param expand: 
    :return: 
    """
    feats = []
    for f in featureExs:
        res = f(info, arg1, arg2)
        if res is not None:
            if type(res) == list:
                for el in res:
                    featStrId = f.__name__ + "#" + el
                    if expand:
                        feats.append(lexicon.getOrAdd(featStrId))
                    else:
                        featId = lexicon.getId(featStrId)
                        if featId is not None:
                            feats.append(featId)
            else:
                featStrId = f.__name__ + "#" + res
                if expand:
                    feats.append(lexicon.getOrAdd(featStrId))
                else:
                    featId = lexicon.getId(featStrId)
                    if featId is not None:
                        feats.append(featId)

    return feats

```python
if args.features == "basic":
    print "Using rich features"
```
rich(`"basic"`, seems the only choice) feature format:
```python
features = [trigger, entityTypes, arg1_lower, arg2_lower, bow_clean, entity1Type, entity2Type, lexicalPattern,
                posPatternPath]
```

# Generate and print args

In [4]:
def getCommandArgs():
    parser = argparse.ArgumentParser(description='Trains a basic Open Information Extraction Model')

    parser.add_argument('--pickled_dataset', metavar='pickled_dataset', nargs='?', required=True,
                        help='the pickled dataset file (produced by OiePreprocessor.py)')

    parser.add_argument('--epochs', metavar='epochs', nargs='?', type=int, default=100,
                        help='maximum number of epochs')

    parser.add_argument('--learning_rate', metavar='learning_rate', nargs='?', type=float, default=0.1,
                        help='initial learning rate')

    parser.add_argument('--batch_size', metavar='batch_size', nargs='?', type=int, default=50,
                        help='size of the minibatches')

    parser.add_argument('--embed_size', metavar='embed_size', nargs='?', type=int, default=30,
                        help='initial learning rate')

    parser.add_argument('--relations_number', metavar='relations_number', type=int, nargs='?', default=3,
                        help='number of relations to induce')

    parser.add_argument('--negative_samples_number', metavar='negative_samples_number', nargs='?', type=int, default=5,
                        help='number of negative samples')

    parser.add_argument('--l1_regularization', metavar='l1_regularization', nargs='?', type=float, default=0.0,
                        help='lambda value of L1 regulatization')

    parser.add_argument('--l2_regularization', metavar='l2_regularization', nargs='?', type=float, default=0.0,
                        help='lambda value of L2 regulatization')

    parser.add_argument('--optimization', metavar='optimization', nargs='?', type=int, default='0',
                        help='optimization algorithm 0 SGD, 1 ADAGrad, 2 ADADelta. Default SDG.')

    parser.add_argument('--model_name', metavar='model_name', nargs='?', required=True, type=str,
                        help='Name or ID of the model')

    parser.add_argument('--model', metavar='model', nargs='?', type=str, required=True,
                        help='Model Type choose among A, C, AC.')

    parser.add_argument('--fixed_sampling', metavar='fixed_sampling', nargs='?', default='False',
                        help='fixed/dynamic sampling switch, default fixed sampling')

    parser.add_argument('--ext_emb', metavar='ext_emb', nargs='?', default='False',
                        help='external embeddings, default False')

    parser.add_argument('--extended_reg', metavar='extended_reg', nargs='?', default='False',
                        help='extended regularization on reconstruction parameters, default False')

    parser.add_argument('--frequent_eval', metavar='frequent_eval', nargs='?', default='False',
                        help='using frequent evaluation, default False')

    parser.add_argument('--seed', metavar='seed', nargs='?', type=int, default=2,
                        help='random seed, default 2')

    parser.add_argument('--alpha', metavar='alpha', nargs='?', type=float, default=1.0,
                        help='alpha coefficient for scaling the entropy term')


    return parser.parse_args(arg_str.split(' '))

def print_args(args):
    tmp_dict = vars(args)
    for item in tmp_dict:
        print "{:30s} : {:10s}".format(item, str(tmp_dict[item]))

In [5]:
arg_str ='--pickled_dataset train.pk --model_name discrete-autoencoder \
--model AC --optimization 1 --epochs 10 --batch_size 100 --relations_number 10 \
--negative_samples_number 5 --l2_regularization 0.1 --alpha 0.1 --seed 2 --embed_size 10 --learning_rate 0.1'

In [6]:
args = getCommandArgs()
print_args(args)

negative_samples_number        : 5         
frequent_eval                  : False     
embed_size                     : 10        
extended_reg                   : False     
l2_regularization              : 0.1       
learning_rate                  : 0.1       
fixed_sampling                 : False     
batch_size                     : 100       
model_name                     : discrete-autoencoder
epochs                         : 10        
optimization                   : 1         
alpha                          : 0.1       
seed                           : 2         
relations_number               : 10        
model                          : AC        
ext_emb                        : False     
pickled_dataset                : train.pk  
l1_regularization              : 0.0       


Translate arguments

In [7]:

rseed = args.seed
rand = np.random.RandomState(seed=rseed)


negativeSamples = args.negative_samples_number
numberRelations = args.relations_number
indexedData, goldStandard = loadData(args, rand, negativeSamples, numberRelations, args.model)


maxEpochs = args.epochs
learningRate = args.learning_rate
batchSize = args.batch_size
embedSize = args.embed_size
lambdaL1 = args.l1_regularization
lambdaL2 = args.l2_regularization
optimization = args.optimization
modelName = args.model_name
model = args.model
fixedSampling = eval(args.fixed_sampling)
extEmb = eval(args.ext_emb)
extendedReg = eval(args.extended_reg)
frequentEval = eval(args.frequent_eval)
alpha = args.alpha
    

Found existing pickled dataset, loading... Done (0.0459129810333s.)
Produced indexed dataset


In [47]:
goldStandard['train'][4] # goldStandard is the relation between the two entities if any (used only for evaluation)

['/business/person/company']

In [25]:
#indexedData # is an instance of OieData.DataSetManager
vars(indexedData).keys()

['id2Arg',
 'negSamplesNum',
 'validExs',
 'featureLex',
 'rng',
 'negSamplingDistr',
 'negSamplingCum',
 'arg2Id',
 'testExs',
 'relationNum',
 'trainExs',
 'negSamplingDistrPower']

In [34]:
print indexedData.negSamplesNum
print len(indexedData.negSamplingCum), indexedData.negSamplingCum[:3]
print len(indexedData.negSamplingDistr), indexedData.negSamplingDistr[:3]
print indexedData.negSamplingDistrPower

5
1476 [ 0.00067751  0.00135501  0.00203252]
1476 [0.0006775067750677507, 0.0006775067750677507, 0.0006775067750677507]
False


In [38]:
print indexedData.trainExs
print indexedData.featureLex


<learning.OieData.MatrixDataSet instance at 0x7ffb39f29b00>
<processing.OiePreprocessor.FeatureLexicon instance at 0x7ffb3a2312d8>


In [ ]:
inducer = ReconstructInducer(indexedData, goldStandard, rand, maxEpochs, learningRate,
                                 batchSize, embedSize, lambdaL1, lambdaL2, optimization, modelName,
                                 model, fixedSampling, extEmb, extendedReg,
                                 frequentEval, alpha)



inducer.learn()

saveModel(inducer, inducer.modelName)